In [ ]:
import sys
sys.path.append('..')

import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from util.preprocessing import extract_month
from util.preprocessing2 import get_temper, get_humid, extract_yyyymmdd, extract_cost1
import re
from dateutil import parser

def preprocess_data2():
    df = pd.read_csv('../data/baccident_data.csv', encoding = 'cp949')
    
    #필요 없는 특성 제거
    allColumns = df.columns
    #{사고데이터:비사고데이터} = {'발생일시':'일자','공공/민간 구분':'발주자구분','기상상태':일자를통해기상청에서불러오기,'공종':'공종','사망자수(명)':0,'부상자수(명)':0,'공사비':'공사비','공사기간':'총공사일수'}
    params = ['일자', '발주자구분', '공종', '공사비', '총공사일수']
    df = df.drop(allColumns.drop(params), axis=1)

    df = df.dropna()
    
    #범주형 데이터를 수치형 데이터로 인코딩
    df['사고발생월'] = df['일자'].apply(extract_month)
    df['공사비'] = df['공사비'].apply(extract_cost1)
    df['일자'] = df['일자'].apply(extract_yyyymmdd)
    
    
    # 기온, 습도 기상청에서 불러오기
    df['기온'] = df['일자'].apply(get_temper)
    df['습도'] = df['일자'].apply(get_humid)
    
    # '사망자수(명)','부상자수(명)' 추가
    df['사망자수(명)'] = 0
    df['부상자수(명)'] = 0
    
    

    # 원-핫 인코딩
    ct = ColumnTransformer([('onehot', OneHotEncoder(sparse=False), ['발주자구분', '공종'])], remainder='passthrough')
    ct.fit(df)
    X = ct.transform(df)

    # 컬럼 이름 리스트 생성
    num_cols = df.columns.tolist()
    ohe = ct.named_transformers_['onehot']
    ohe_cols = ohe.get_feature_names_out(['발주자구분', '공종']).tolist()
    new_cols = ohe_cols + num_cols
    new_cols.remove('발주자구분')
    new_cols.remove('공종')

    # DataFrame으로 변환
    df = pd.DataFrame(X, columns=new_cols)

    df = df.astype('float64')

    df.to_csv("ba_ouput.csv")

preprocess_data2()